In [ ]:
import win32com.client
import configparser
import pandas as pd
from datetime import datetime,timedelta
import pytz

config = configparser.ConfigParser()
config.sections()
config.read('PIMX_Config.ini')

In [ ]:
try:
    pitags_df = pd.read_csv(config['FilePath']['TAG_INPUT_FILE'])
    pitags_df.head()
except Exception as e:
    print("Exception occurred", exc_info=e)

In [ ]:
try:
    pi_sdk = win32com.client.Dispatch('PISDK.PISDK')
    conn = win32com.client.Dispatch('PISDKDlg.Connections')
    pi_server = pi_sdk.Servers(config['PI_CFG']['SERVER'])
    conn.Login(pi_server,config['PI_CFG']['USER'],'',1,0) 
except Exception as e:
    print("Exception occurred", exc_info=e)

In [ ]:
current_time = pytz.timezone("Asia/Bangkok").localize(datetime.now())
start_time = (current_time - timedelta(hours=1)).strftime("%d-%b-%y %H:10").lower()
end_time = current_time.strftime("%d-%b-%y %H:00").lower()
# current_time = pytz.timezone("Asia/Bangkok").localize(datetime.now())
# interval = 5
# delta_minute = int(current_time.strftime('%M')) % interval
# start_time = (current_time - timedelta(minutes=interval + delta_minute)).strftime("%d-%b-%y %H:%M").lower()
# end_time = (current_time - timedelta(minutes=delta_minute)).strftime("%d-%b-%y %H:%M").lower()
print('read data from : {} to {}'.format(start_time,end_time))

In [ ]:
try:
    rows_to_insert = []
    for index,row in pitags_df.iterrows():
        pi_point = pi_server.PIPoints[row['PITags']] 
        recorded_values = pi_point.Data.InterpolatedValues('01-dec-21 08:10','01-dec-21 12:00',24,"",2,None)
        if recorded_values.Count > 1 :
            for raw_index,raw_data in enumerate(recorded_values) :
                Value=raw_data.Value
                rows_to_insert.append({u"timestamp" : raw_data.TimeStamp.LocalDate.strftime("%Y-%m-%dT%H:%M:%S+07:00"),
                       u"id" : row['PITags'],
                       u"value": Value,
                       u"created_at" : pytz.timezone("Asia/Bangkok").localize(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S+07:00")
                       })
                # rows_to_insert.append((raw_data.TimeStamp.LocalDate.strftime("%Y-%m-%dT%H:%M:%S+07:00"),
                #        row['PITags'],
                #        Value,
                #         pytz.timezone("Asia/Bangkok").localize(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S+07:00")
                #        ))
except Exception as e:
    print("Exception occurred", exc_info=e)
# print(sorted(rows_to_insert,key=lambda x: x['timestamp']))
print('prepare data : {} rows'.format(len(rows_to_insert)))

In [ ]:
# client = bigquery.Client()
# errors = client.insert_rows_json('merlindb.RAW_SENSOR', rows_to_insert)  # Make an API request.
# if errors == []:
#     print("New rows have been added.")
# else:
#     print("Encountered errors while inserting rows: {}".format(errors))

In [ ]:
import pymssql

server = 'server'
user = 'user'
password = 'pwd'
db= 'db'
conn = pymssql.connect(server, user, password, db)
cursor = conn.cursor()
cursor.executemany(
    "INSERT INTO SENSOR_TAG VALUES (%d, %s, %s)",rows_to_insert)
# you must call commit() to persist your data if you don't set autocommit to True
conn.commit()

# cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')
# row = cursor.fetchone()
# while row:
#     print("ID=%d, Name=%s" % (row[0], row[1]))
#     row = cursor.fetchone()

conn.close()